# to generate illumina nex and ht sample sheet with barcodes

# nex

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from io import StringIO
import os
import csv
import sys
import glob
from collections import OrderedDict,defaultdict,Counter

%matplotlib inline 

import xlsxwriter
from scipy import stats

import seaborn as sns
# from pandas.tools import plotting
from matplotlib import pyplot as plt


from Bio.Alphabet import generic_dna,generic_rna,generic_protein
import primer3
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import math
import pysam



In [17]:
 !pip3 install primer3-py

    100% |████████████████████████████████| 552kB 22.7MB/s ta 0:00:01


In [2]:
current_folder_path, current_folder_name = os.path.split(os.getcwd())


In [3]:
current_folder_path

'/Users/ltao/Projects/glioma/exp/190405'

In [4]:
# define the name of the directory to be created
samples_folder = current_folder_path+'/samples'

try:  
    os.mkdir(samples_folder)
except OSError:  
    print ("Creation of the directory %s failed" % samples_folder)
else:  
    print ("Successfully created the directory %s " % samples_folder)

Successfully created the directory /Users/ltao/Projects/glioma/exp/190405/samples 


In [40]:
# try:  
#     os.rmdir(path)
# except OSError:  
#     print ("Deletion of the directory %s failed" % path)
# else:  
#     print ("Successfully deleted the directory %s" % path)

Successfully deleted the directory /Users/ltao/Projects/glioma/exp/190402/tst


In [6]:
samplesheet_file='/Users/ltao/Projects/Data/miseq/nex/MS8001943-300V2.csv'
sample_sheet = pd.read_csv(samplesheet_file)
input_key = sample_sheet.iloc[[19]].values.tolist()

In [8]:
sheets = pd.read_excel('/Users/ltao/Projects/Data/miseq/nex/Nextra-Ads.xlsx', sheet_name=None)
nexads=dict()
for sheet_name, sheet in sheets.items():
    print (sheet_name)
nex_i5=sheets["plate AD1-i5"]
nex_i5.set_index('96 Well Columns', inplace=True)
nex_i7=sheets["plate Ad2 i7"]
nex_i7.set_index('96 Well Columns', inplace=True)
layout_i5=sheets['layout_i5']
layout_i7=sheets['layout_i7']

plate AD1-i5
plate Ad2 i7
layout_i5
layout_i7
Fill Along Columns
Sheet1


In [9]:
def primer_loc(rows,cols):
    locs=list()
    for r5 in rows:
        for c5 in cols:
             locs.append(str(r5)+str(c5))
    return locs

In [10]:
### combine i5, i7 by rows: A:A; B:B....H:H
def xbarcodes_pairs(rows_i5='ABCD',rows_i7='ABCD',cols_i5='5678',cols_i7='5678'):
    barcodes_i5=primer_loc(rows_i5,cols_i5)
    barcodes_i7=primer_loc(rows_i7,cols_i7)
    xbarcodes_i5=list()
    for j in barcodes_i5:
        for i in range(len(cols_i7)):
            xbarcodes_i5.append(j)
    barcodes_pair=list(zip(xbarcodes_i5,barcodes_i7))
    return barcodes_pair
#     return xbarcodes_i5

In [11]:

# non-dig-hcc38-0.7xbeads:

# Ad1: col 11
# Ad2:1-6


rows_i5='ABCDEFGH'
rows_i7='ABCDEFGH'
cols_i5=[11,]
cols_i7=[1,2,3]

hcc38_rtl_non_dig=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)


# MGH
cols_i5=[9,]
cols_i7=[1,2,3,4]
# cols_i7=[7,8,9,10,11,12]

mgh_5_gt=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[2,]
cols_i7=[1,2,3]

hcc38_rtl=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[2,]
cols_i7=[4,5,6]

hcc38_pbs=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[3,]
cols_i7=[7,8,9]


hcc38_pbs_beads=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

In [12]:
nex_i5_dic=nex_i5.to_dict(orient="index")
nex_i7_dic=nex_i7.to_dict(orient="index")

In [13]:
input_key = sample_sheet.iloc[[19]].values.tolist()[0]
input_bp=OrderedDict()


In [14]:
### chose runtype
color = int(input("NovaSeq, MiSeq, HiSeq 2000/2500 == four_color, \n MiniSeq, NextSeq, HiSeq 3000/4000 == two_color, \n what is your run? \n pls input 2 or 4\n"))

if color == 4:
    runtype ="four_color"
    print ("your run is ###four_color###")
elif color == 2:
    runtype ="two_color"
    print ("your run is ###two_color###")
else:
    print ('pls define your runtype, 2 or 4')

NovaSeq, MiSeq, HiSeq 2000/2500 == four_color, 
 MiniSeq, NextSeq, HiSeq 3000/4000 == two_color, 
 what is your run? 
 pls input 2 or 4
4
your run is ###four_color###


In [15]:
## for NovaSeq, MiSeq, HiSeq 2000/2500, 
for plate,bps in {"hcc38_rtl_non_dig":hcc38_rtl_non_dig,"mgh_5_gt":mgh_5_gt,"hcc38_rtl":hcc38_rtl,"hcc38_pbs":hcc38_pbs,"hcc38_pbs_beads":hcc38_pbs_beads}.items():
    for bp in bps:
        samples_input_dict=OrderedDict()
        if runtype == "four_color":
            i5_barcodes = nex_i5_dic[bp[0]]['i5 Bases for Sample Sheet NovaSeq, MiSeq, HiSeq 2000/2500\xa0']
        else:
            i5_barcodes = nex_i5_dic[bp[0]]['i5 Bases for Sample Sheet MiniSeq, NextSeq, HiSeq 3000/4000 ']

        i7_barcodes = nex_i7_dic[bp[1]]['on Sample Sheet']
        for k in input_key:
            samples_input_dict[k] = 'i7_'+bp[1]+'_i5_'+bp[0]
        samples_input_dict['index2'] =i5_barcodes
        samples_input_dict['index'] =i7_barcodes
        samples_input_dict['Sample_Plate'] = plate
        input_bp['i7_'+bp[1]+'_i5_'+bp[0]]=samples_input_dict


In [16]:
barcodes_file=samples_folder+'/barcodes.xlsx'

input_bp_df=pd.DataFrame.from_dict(input_bp, orient='index')
input_bp_df.to_excel(barcodes_file)

In [20]:
! cp /Users/ltao/Projects/Data/miseq/nex/MS8001943-300V2.csv ../samples/samplesheet.csv

In [24]:
xxx=1
first_run=input('is this the first time run this cell? yes or no?')
if first_run=='yes' or first_run=='y':
    fff=1
else:
    fff=0
xxx=xxx-fff
if xxx==0:

    fastqs_folder=current_folder_path+'/fastqs/'
    os.mkdir(fastqs_folder)
    xxx+=1
else:
    print ("NOT the first run")

is this the first time run this cell? yes or no?n
not the first run


### Prepare all fastqs by shell in merge_script


In [29]:
os.listdir(fastqs_folder)

['i7_B2_i5_B9_S22_L001_R2_001.fastq',
 'i7_B7_i5_B3',
 'i7_D4_i5_D9_S59_L001_R1_001.fastq',
 'i7_B3_i5_B9_S25_L001_R2_001.fastq',
 'i7_B2_i5_B2_S21_L001_R1_001.fastq',
 'i7_H5_i5_H2_S124_L001_R2_001.fastq',
 'i7_F1_i5_F11_S81_L001_R2_001.fastq',
 'i7_D4_i5_D9',
 'i7_A8_i5_A3',
 'i7_A3_i5_A9',
 'i7_G2_i5_G2',
 'i7_B5_i5_B2',
 'i7_C2_i5_C2_S37_L001_R1_001.fastq',
 'i7_E9_i5_E3_S80_L001_R2_001.fastq',
 'i7_G2_i5_G11',
 'i7_D1_i5_D9_S51_L001_R2_001.fastq',
 'i7_H4_i5_H2',
 'i7_E1_i5_E9',
 'i7_D2_i5_D11',
 'i7_F5_i5_F2',
 'i7_H2_i5_H2_S117_L001_R1_001.fastq',
 'i7_H6_i5_H2_S125_L001_R2_001.fastq',
 'i7_C2_i5_C2',
 'i7_E3_i5_E9',
 'i7_D3_i5_D9_S57_L001_R1_001.fastq',
 'i7_H6_i5_H2',
 'i7_E8_i5_E3',
 'i7_A1_i5_A9',
 'i7_C1_i5_C9_S35_L001_R1_001.fastq',
 'i7_D2_i5_D2_S53_L001_R2_001.fastq',
 'i7_H1_i5_H11_S113_L001_R1_001.fastq',
 'i7_C3_i5_C9_S41_L001_R1_001.fastq',
 'i7_G2_i5_G11_S100_L001_R1_001.fastq',
 'i7_A3_i5_A2_S8_L001_R1_001.fastq',
 'i7_E1_i5_E9_S67_L001_R1_001.fastq',
 'i7_F7_i5_F3

In [30]:

merge_next_file=fastqs_folder+"merge_nex.txt"
# prepare merge config files
fastqs=list()
for x in os.listdir(fastqs_folder):
    if 'fastq' in x:
        fastqs.append(x)
fastqs=sorted(fastqs
             )
merge_key=['sample','r1','r2']
merge_nex=OrderedDict()

for x in zip(fastqs[::2],fastqs[1::2]):
    pe_reads=dict()
    sindex=x[0].index('_S')
    name=x[0][:sindex]
#     print (name)
    mk=input_bp[name]["Sample_Plate"]+"_"+name
    pe_reads['sample']=mk
    pe_reads ['r1']=x[0]
    pe_reads['r2']=x[1]
    merge_nex[mk]=pe_reads
df_merge_nex=pd.DataFrame.from_dict(merge_nex, orient='index')
df_merge_nex.to_csv(merge_next_file,sep="\t")  
# merge_nex

In [31]:
merge_nex

OrderedDict([('hcc38_rtl_non_dig_i7_A1_i5_A11',
              {'sample': 'hcc38_rtl_non_dig_i7_A1_i5_A11',
               'r1': 'i7_A1_i5_A11_S1_L001_R1_001.fastq',
               'r2': 'i7_A1_i5_A11_S1_L001_R2_001.fastq'}),
             ('hcc38_rtl_i7_A1_i5_A2',
              {'sample': 'hcc38_rtl_i7_A1_i5_A2',
               'r1': 'i7_A1_i5_A2_S2_L001_R1_001.fastq',
               'r2': 'i7_A1_i5_A2_S2_L001_R2_001.fastq'}),
             ('hcc38_rtl_non_dig_i7_A2_i5_A11',
              {'sample': 'hcc38_rtl_non_dig_i7_A2_i5_A11',
               'r1': 'i7_A2_i5_A11_S4_L001_R1_001.fastq',
               'r2': 'i7_A2_i5_A11_S4_L001_R2_001.fastq'}),
             ('hcc38_rtl_i7_A2_i5_A2',
              {'sample': 'hcc38_rtl_i7_A2_i5_A2',
               'r1': 'i7_A2_i5_A2_S5_L001_R1_001.fastq',
               'r2': 'i7_A2_i5_A2_S5_L001_R2_001.fastq'}),
             ('mgh_5_gt_i7_A2_i5_A9',
              {'sample': 'mgh_5_gt_i7_A2_i5_A9',
               'r1': 'i7_A2_i5_A9_S6_L001_R1_001.fast

### Go to merge shell script 

In [ ]:
# allele dropout analysis

In [49]:
cd ../fastqs

/Users/ltao/Projects/glioma/exp/190405/fastqs


In [43]:

mfasta = [f for f in os.listdir(fastqs_folder) if f.endswith('MERGED')]
mfasta=sorted(mfasta, reverse=True)

In [57]:
def rank_amplicon(merged_fasta):

    records = list(SeqIO.parse(merged_fasta, "fasta"))
    s=list()

    for rcd in records:
        s.append((str(rcd.seq.lower()),rcd.id))
    
    unique_reads = defaultdict(list)
    for k, v in s:
        unique_reads[k].append(v)
    rst=dict()
    rst['total_reads']=len(records)

    for k,v in unique_reads.items():
        rst[k]=len(v)
#     return rst
    return unique_reads

In [54]:
# Python lib for interacting with BLAT server


"""
	BLAT server info:
		gfServer start localhost 8123 canFam3.2bit &
		gfServer start localhost 8124 hg19.2bit &
		gfServer start localhost 8125 mm10.2bit &
		gfServer -trans -mask start localhost 8126 canFam3.2bit &
		gfServer -trans -mask start localhost 8127 hg19.2bit &
		gfServer -trans -mask start localhost 8128 mm10.2bit &
"""

class BlatClient:
	def __init__(self, host_name, port_number, is_protein_bool):
		self.host = host_name
		self.port = port_number
		self.is_protein = is_protein_bool

	def blat_sequence(self, nuc_string=""):
		"""
			Take a nucleotide sequence. Returns a list of BlatAlignment objects. 
		"""	
		blat_alignments = []

		to_blat = ">blatter\n%s" % nuc_string
		blat_results = os.popen("echo '%s' | /Users/ltao/biotools/blat/gfClient %s %i -minScore=0 -minIdentity=0 /Users/ltao/biotools/blat/ stdin stdout" % (to_blat ,self.host, self.port)).read().split("\n")
		for result in blat_results:
			spl = result.split("\t")
			if spl[0].isdigit():			
				algn = BlatAlignment(result,self.is_protein)
				blat_alignments.append(algn)

		return blat_alignments

class BlatAlignment:
	"""
		PSL format line and returns a BlatAlignment object
	"""
	def __init__(self, psl_line, is_protein_bool):
		matches,misMatches,repMatches,nCount,qNumInsert,qBaseInsert,tNumInsert,tBaseInsert,strand,qName,qSize,qStart,qEnd,tName,tSize,tStart,tEnd,blockCount,blockSizes,qStarts,tStarts = psl_line.split("\t")
		
		self.start = int(tStart.strip())
		self.stop = int(tEnd.strip())
		self.chromosome = tName.strip()
		self.strand = strand.strip()
		self.base_matches = int(matches.strip())
		self.mismatches = int(misMatches.strip())
		self.repeat_matches = int(repMatches.strip())
		self.query_size = int(qSize.strip())
		self.pct_match = float(self.base_matches) / float(self.query_size)
		self.span = self.stop - self.start
		self.num_gaps = int(tNumInsert.strip())
		self.total_gap_size = int(tBaseInsert.strip()) 
		self.n_masked_bases = int(nCount.strip())
		self.query_start = int(qStart.strip())
		self.query_stop = int(qEnd.strip())
		self.query_gap_count = int(qNumInsert.strip())
		self.query_gap_bases = int(qBaseInsert.strip())
		self.block_count = int(blockCount.strip())
		self.block_sizes = [int(x.strip()) for x in blockSizes.strip().split(",") if x != ""]
		self.query_block_starts = [int(x.strip()) for x in qStarts.strip().split(",") if x != ""]
		self.block_starts = [int(x.strip()) for x in tStarts.strip().split(",") if x != ""]
		self.is_protein = is_protein_bool

	def get_pct_ident(self):
		"""
			Calculates UCSCs percent identity score
		"""
		if self.is_protein:
			sizeMul = 3
		else:
			sizeMul = 1

		# Calculate alignment size
		qAliSize = sizeMul * (self.query_stop - self.query_start)
		tAliSize = self.stop - self.start
		aliSize = min(qAliSize, tAliSize)

		# Break if something fucked up
		if aliSize <= 0:
			return 0

		# Get difference in alignment size
		sizeDif = qAliSize - tAliSize
		if sizeDif < 0:
			sizeDif = -sizeDif
		
		insertFactor = self.query_gap_count

		total = (sizeMul * (self.base_matches + self.repeat_matches + self.mismatches))
		if total != 0:
			milliBad = (1000 * (self.mismatches * sizeMul + insertFactor + round(3 * math.log10(1 + sizeDif)))) / total
		
		pct_ident = 100.0 - milliBad * 0.1
		
		return pct_ident




In [58]:
counts=dict()
for mfa in mfasta:
    counts[mfa]=rank_amplicon(mfa)

In [72]:
for k in list(counts.keys()):
    print (k)
    i=counts[k]
    tmp=dict()
    for kk,ii in i.items():
        tmp[kk]=len(ii)
    from collections import OrderedDict
    dd = OrderedDict(sorted(tmp.items(), key=lambda x: x[1],reverse=True))
    counts[k]["ranking"]=dd
    for seq in list(dd.keys())[:10]:
        print (seq, type(seq),dd[seq])

mgh_5_gt_i7_H1_i5_H9_MERGED
ranking <class 'str'> 1
mgh_5_gt_i7_G3_i5_G9_MERGED
ranking <class 'str'> 1
mgh_5_gt_i7_G1_i5_G9_MERGED
ranking <class 'str'> 1
mgh_5_gt_i7_F3_i5_F9_MERGED
ranking <class 'str'> 1
mgh_5_gt_i7_F2_i5_F9_MERGED
ranking <class 'str'> 2
acaatctcggctcactgcaacctctgcctcccgggttcacaccattctcctgcctcagcctctcgagtagctgggactccaggtgcccgccaccacgcccagctaattttttgtatttttagtagagacagggtttcaccttgttagccaagatggtctcgatcttctgacctc <class 'str'> 1
mgh_5_gt_i7_E3_i5_E9_MERGED
ranking <class 'str'> 1
mgh_5_gt_i7_E1_i5_E9_MERGED
ranking <class 'str'> 1
mgh_5_gt_i7_D4_i5_D9_MERGED
ranking <class 'str'> 2
gaggattggagacattgggtacaagaagatactttgtaagtttatgaatgaattctgctctatgtaagatgatagactaggtttttggaggtaacactatcaaacaagtatattggatgtatatgtatatgtgtgtgtgtgtgtgtgcatatgtgtatatgtgtgtatatactgtcagccgtttgcatctgggagtcaa <class 'str'> 1
mgh_5_gt_i7_D3_i5_D9_MERGED
ranking <class 'str'> 1
mgh_5_gt_i7_D1_i5_D9_MERGED
ranking <class 'str'> 1
mgh_5_gt_i7_C4_i5_C9_MERGED
ranking <class 'str'> 1
mgh_5_gt_i7_C3_i5_C9_

In [74]:
seq

'agaaatgtgagttccatgcaagattaattgaatcaggaataagcatctggctgctgggtggcatgagtgaaatatagtcaactgtgctgactttctcttataaacatcttttcataacagatactgtggcaagatttttccaaggtctgcaaacctaacacggcacttgagaacccacacaggagagcagccttacaggtctgacaatgatatttgct'

In [75]:
alns=blatter.blat_sequence(seq)

for a in alns:
    print (a.chromosome,a.start,a.stop,a.strand)
    print ("Base Matches",a.base_matches)
    print ("Base mismatches",a.mismatches)
    print ("Percent Match",a.pct_match)
    print ("Percent Identitty",a.get_pct_ident())


#  in shell run "/Users/ltao/biotools/blat/gfServer start localhost 8124 /Users/ltao/biotools/blat/hg19.2bit"

In [64]:
blatter = BlatClient('localhost',8124,False)

for k in list(counts.keys())[:50]:
    print (k)
    dd=counts[k]['ranking']
    for seq in list(dd.keys()):
#         dd['blat']=blatter.blat_sequence(seq)
        alns=blatter.blat_sequence(seq)
        tmp=dict()
        ttk=0
#         for a in alns:
#             ttk+=1
#             ttmp=dict()
#             ttmp['target']= (a.chromosome,a.start,a.stop,a.strand)
#             ttmp ["Base Matches"]= a.base_matches
#             ttmp["Base mismatches"]=a.mismatches
#             ttmp["Percent Match"]=a.pct_match
#             ttmp["Percent Identitty"]=a.get_pct_ident()
#             tmp[ttk]=ttmp
#         dd['alns']=tmp
#         for a in alns:
#             print (a.chromosome,a.start,a.stop,a.strand)
#             print ("Base Matches",a.base_matches)
#             print ("Base mismatches",a.mismatches)
#             print ("Percent Match",a.pct_match)
#             print ("Percent Identitty",a.get_pct_ident())
#         print ('''\n''')
        print (alns)

mgh_5_gt_i7_H1_i5_H9_MERGED
mgh_5_gt_i7_G3_i5_G9_MERGED
mgh_5_gt_i7_G1_i5_G9_MERGED
mgh_5_gt_i7_F3_i5_F9_MERGED
mgh_5_gt_i7_F2_i5_F9_MERGED
[]
[]
mgh_5_gt_i7_E3_i5_E9_MERGED
mgh_5_gt_i7_E1_i5_E9_MERGED
mgh_5_gt_i7_D4_i5_D9_MERGED
[]
[]
mgh_5_gt_i7_D3_i5_D9_MERGED
mgh_5_gt_i7_D1_i5_D9_MERGED
mgh_5_gt_i7_C4_i5_C9_MERGED
mgh_5_gt_i7_C3_i5_C9_MERGED
mgh_5_gt_i7_C2_i5_C9_MERGED
mgh_5_gt_i7_C1_i5_C9_MERGED
mgh_5_gt_i7_B3_i5_B9_MERGED
mgh_5_gt_i7_B2_i5_B9_MERGED
mgh_5_gt_i7_B1_i5_B9_MERGED
mgh_5_gt_i7_A4_i5_A9_MERGED
mgh_5_gt_i7_A2_i5_A9_MERGED
hcc38_rtl_non_dig_i7_H3_i5_H11_MERGED
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
hcc38_rtl_non_dig_i7_H2_i5_H11_MERGED
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
hcc38_rtl_non_dig_i7_H1_i5

In [58]:

counts_df=pd.DataFrame.from_dict(counts, orient='index')
counts_df.to_excel(path+'mecon_adr_counts0.1.xlsx')

# vcf files to matrics


In [60]:
vcfs=list()
path='/Users/ltao/Projects/Data/miseq/{0}/fastqs/'.format(flow_cell_id)
for x in os.listdir(path):
    if 'vcf' in x:
        vcfs.append(x)

In [63]:
for vcf in vcfs:
    vcf = pysam.AlignmentFile(vcf, "rb", check_sq=False)
    for read in vcf.fetch('chr1'):
        print (read.get_reference_sequence()," \n", read.seq, read.cigar)
    break
                



ValueError: file does not contain alignment data